In [1]:
using Random, Combinatorics, LinearAlgebra, Convex, Mosek, BenchmarkTools, DelimitedFiles, StatsBase
include("functions.jl")

function brute(X, k; μ=0.1)
    n = size(X)[1]
    cand = collect(combinations(1:n, k))
    val = zeros(length(cand))
    for i in 1:length(cand)
        X0 = X[cand[i],:]
        val[i] =  tr(X*((X0'X0+μ*I)\(X')))
    end
    return cand[findmin(val)[2]]
end

function brute2(X, k; μ=0.1)
    n = size(X)[1]
    cand = collect(combinations(1:n, k))
    best = 0
    val = Inf
    for i in 1:length(cand)
        X0 = X[cand[i],:]
        if tr(X*((X0'X0+μ*I)\(X'))) < val
            val = tr(X*((X0'X0+I)\(X')))
            best = i
        end
    end
    return cand[best]
end

function alg1(X, k; μ=0.1)
    K = X*X'
    n = size(X)[1]
    list = zeros(Int64, k)
    for i = 1:k
        val = zeros(n)
        for j = 1:n
            val[j] = sum(abs2, K[:,j]) / (K[j,j]+μ)
        end
        list[i] = findmax(val)[2]
        K -= 1/(K[list[i],list[i]]+μ)*K[:,list[i]]*K[:,list[i]]'
    end
    return list
end

function alg3(X,k;μ=0.1, γ=1)
    K = X*X'
    n = size(X)[1]

    α = zeros(n,n)
    β =  ones(n)
    label_old = label_new =  zeros(Int64, k) 
    cnt = 0
    
    while cnt < 10
        for i = 1:n
            α[:,i] = (Diagonal(1 ./ β) + K)\K[:,i]
        end

        for j = 1:n
            β[j] = sqrt(1/γ * sum(abs2, α[j,:]))
        end

        label_new = sortperm(β, rev=true)[1:k]
        if label_new == label_old
            cnt += 1
        else
            cnt = 0
        end
        label_old = copy(label_new)
    end
    
    return label_new
end

function alg4(X,k;μ=0.1, γ=1)
    K = X*X'
    n = size(X)[1]

    α = αnew = zeros(n,n)
    β = βnew =  ones(n)
    cnt = 0
    
    while cnt < 10
        for i = 1:n
            αnew[:,i] = (Diagonal(1 ./ β) + K)\K[:,i]
        end

        for j = 1:n
            βnew[j] = sqrt(1/γ * sum(abs2, α[j,:]))
        end

        if norm(α-αnew) + norm(β-βnew) < 10^-13
            break
        end
        α=αnew
        β=βnew
    end
    
    return sortperm(βnew)[1:k]
    #return norm(βnew), sort(sortperm(βnew)[1:k])
end

alg4 (generic function with 1 method)

In [ ]:
Random.seed!(1992)
X = randn(50,4)

μ = 0.1
n, p = size(X)
k = 5
println(brute(X,k))
println(findall(x->x>0.5,sagnol_A(X, μ*Diagonal(ones(p)), k; K=X', verbose=0, IC=1)))
println(sort(alg1(X,k)))
println(sort(alg3(X,k)))
println(sort(alg4(X,k)))

In [ ]:
X = randn(50,4)
n, p = size(X)
μ = 0.1
k = 5
println(brute(X,k))
println(findall(x->x>0.5,sagnol_A(X, μ*Diagonal(ones(p)), k; K=X', verbose=0, IC=1)))
println(sort(alg1(X,k)))
println(sort(alg3(X,k)))
println(sort(alg4(X,k)))

In [2]:
Random.seed!(1992)
wifi = readdlm("trainingData.csv",',',Any,'\n')
colnames = wifi[1,:]
wifi = wifi[2:end,1:end-1]
n, p = size(wifi)
wifi = wifi[sample(1:n, 100, replace = false),:]
y = convert(Array{Float64,1}, wifi[:,523])
X = convert(Array{Float64, 2}, wifi[:,1:200])
n, p = size(X)
μ = 0.1
k = 50

50

In [6]:
@time findall(x->x>0.5, sagnol_A(X, μ*Diagonal(ones(p)), k; K=X', verbose=1, IC=0))

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 50508           
  Cones                  : 101             
  Scalar variables       : 60506           
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator - tries                  : 0                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.08            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.75    
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 50508           
  Cones               

84-element Array{Int64,1}:
   1
   2
   5
   6
   7
   8
   9
  10
  12
  13
  14
  15
  16
   ⋮
  88
  89
  90
  91
  92
  93
  94
  95
  97
  98
  99
 100

In [ ]:
@time findall(x->x>0.5, sagnol_A(X, μ*Diagonal(ones(p)), k; K=X', verbose=1, IC=1))

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 50407           
  Cones                  : 101             
  Scalar variables       : 60506           
  Matrix variables       : 0               
  Integer variables      : 101             

Optimizer started.
Mixed integer optimizer started.
Threads used: 24
Presolve started.
Presolve terminated. Time = 3.94
Presolved problem: 30402 variables, 20202 constraints, 4040702 non-zeros
Presolved problem: 0 general integer, 100 binary, 30302 continuous
Clique table size: 0
BRANCHES RELAXS   ACT_NDS  DEPTH    BEST_INT_OBJ         BEST_RELAX_OBJ       REL_GAP(%)  TIME  
0        1        0        0        NA                   -5.0003203802e-03    NA          18.1  
0        1        0        0        -5.4789156434e-08    -5.0003203802e-03    9.13e+06    166.4 
Cut generation started.
0        2        0      

50-element Array{Int64,1}:
  4
  8
  9
 10
 12
 14
 19
 21
 24
 25
 26
 30
 32
  ⋮
 80
 83
 85
 86
 88
 90
 92
 93
 94
 97
 98
 99

In [3]:
@time sort(alg1(X,k))

  1.492842 seconds (2.37 M allocations: 129.220 MiB)


50-element Array{Int64,1}:
   2
   4
   7
   8
   9
  10
  11
  14
  16
  17
  20
  25
  26
   ⋮
  77
  80
  81
  82
  89
  90
  91
  93
  94
  95
  96
 100

In [4]:
@time sort(alg3(X,k))

122.967789 seconds (5.03 M allocations: 415.595 MiB)


50-element Array{Int64,1}:
   1
   5
   6
  10
  12
  13
  14
  15
  20
  21
  23
  25
  28
   ⋮
  76
  77
  80
  83
  86
  89
  91
  92
  93
  94
  98
 100

In [5]:
@time sort(alg4(X,k))

 12.419969 seconds (796.03 k allocations: 54.642 MiB, 1.99% gc time)


50-element Array{Int64,1}:
  2
  3
  4
  7
  8
  9
 11
 16
 17
 18
 19
 22
 24
  ⋮
 79
 81
 82
 84
 85
 87
 88
 90
 95
 96
 97
 99

In [ ]:
for i = 10 .^ convert.(Float64, collect((-10:10)))
    println(sort(alg4(X,k,γ=i)))
end

# lasso path does not cross!

In [ ]:
@benchmark alg3(X,k)

In [ ]:
@benchmark alg4(X,k)

In [ ]:
K = X*X'
n = size(X)[1]
d, P = eigen(K)
rk = rank(X)
d[1:n-rk] .= 0
γ = 10

α = zeros(n,n)
β = βinv = ones(n)

# step 1 
for i = n-rk+1:n
    α[:,i] = d[i]^(3/2)/(d[i]^2+μ*d[i])*(βinv .* P[:,i])
end

# step 2
t = Variable(n)
β_var = Variable(n)
problem = minimize(sum(t) + γ*sum(β_var))
for i = 1:n
    problem.constraints += t >= sumsquares(sqrt(d[i])*P[:,i] - K*Diagonal(α[:,i])*β_var) + μ*d[i]*sumsquares(Diagonal(α[:,i])*β_var)
end
problem.constraints += β_var >= 0
solve!(problem, MosekSolver(LOG=0))
β̂ = vec(β_var.value)
β̂[findall(x->x<10^-6, β̂)] .= 0
println(norm(β̂))

#println(norm(β-β̂))

# step 3
β .*= β̂
println(norm(β))
βinv = copy(β)
βinv[findall(x->x<10^-6, βinv)] .= Inf
βinv = 1 ./ βinv
println(norm(βinv))
sortperm(β, rev = true)[1:k]

In [ ]:
# step 1 
for i = n-rk+1:n
    α[:,i] = d[i]^(3/2)/(d[i]^2+μ*d[i])*(βinv .* P[:,i])
end

# step 2
t = Variable(n)
β_var = Variable(n)
problem = minimize(sum(t) + γ*sum(β_var))
for i = 1:n
    problem.constraints += t >= sumsquares(sqrt(d[i])*P[:,i] - K*Diagonal(α[:,i])*β_var) + μ*d[i]*sumsquares(Diagonal(α[:,i])*β_var)
end
problem.constraints += β_var >= 0
solve!(problem, MosekSolver(LOG=0))
β̂ = vec(β_var.value)
β̂[findall(x->x<10^-6, β̂)] .= 0
println(norm(β̂))

#println(norm(β-β̂))

# step 3
β .*= β̂
#β = copy(β̂)
println(norm(β))
βinv = copy(β)
βinv[findall(x->x<10^-6, βinv)] .= Inf
βinv = 1 ./ βinv
println(norm(βinv))
sortperm(β, rev = true)[1:k]

In [ ]:
function alg2(X,μ,k)
 x
end